<a href="https://colab.research.google.com/github/sabill/MLCC-Project/blob/master/first_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Coba data yg klasifikasi (joint 2019 dr kaggle)

In [0]:
#@title Run on TensorFlow 2.x
%tensorflow_version 2.x

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from matplotlib import pyplot as plt